<a href="https://colab.research.google.com/github/rahulroynit/Bayesian-vs-Hyperband-Optimization/blob/master/BH_Regression_(Hyperband_vs_Bayesian).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# create set up for installing kerastuner
# installation needed only once
import os, sys
lib_path = '/content/libraries'
os.symlink('/content/drive/My Drive/Colab Notebooks', lib_path)
sys.path.insert(0, lib_path)

In [ ]:
# install keras-tuner
# needs to be run once for permanent installation in colab
!pip install --target=$lib_path keras-tuner

In [3]:
# import dependencies
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import Hyperband, BayesianOptimization
from kerastuner import HyperModel
from keras.datasets import boston_housing 
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import datetime
%load_ext tensorboard
pd.set_option('display.max_columns', None)

Using TensorFlow backend.


In [4]:
# load dataset into train and evaluation data
# evaluation data will be split into test and valid data
(X_train, y_train), (X_eval, y_eval) = boston_housing.load_data()

57344/57026 [==============================] - 0s 2us/step


In [5]:
# query train data
print(type(X_train), type(y_train))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [6]:
# check dims of train and eval data
print('dims of train and eval data')
print(X_train.shape, X_eval.shape)

dims of train and eval data
(404, 13) (102, 13)


In [7]:
# check train data info
print(pd.DataFrame(X_train).info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       404 non-null    float64
 1   1       404 non-null    float64
 2   2       404 non-null    float64
 3   3       404 non-null    float64
 4   4       404 non-null    float64
 5   5       404 non-null    float64
 6   6       404 non-null    float64
 7   7       404 non-null    float64
 8   8       404 non-null    float64
 9   9       404 non-null    float64
 10  10      404 non-null    float64
 11  11      404 non-null    float64
 12  12      404 non-null    float64
dtypes: float64(13)
memory usage: 41.2 KB
None


In [8]:
# query train data samples
print(pd.DataFrame(X_train).head())
print(pd.DataFrame(y_train).head())

         0     1      2    3      4      5      6       7     8      9    10  \
0  1.23247   0.0   8.14  0.0  0.538  6.142   91.7  3.9769   4.0  307.0  21.0   
1  0.02177  82.5   2.03  0.0  0.415  7.610   15.7  6.2700   2.0  348.0  14.7   
2  4.89822   0.0  18.10  0.0  0.631  4.970  100.0  1.3325  24.0  666.0  20.2   
3  0.03961   0.0   5.19  0.0  0.515  6.037   34.5  5.9853   5.0  224.0  20.2   
4  3.69311   0.0  18.10  0.0  0.713  6.376   88.4  2.5671  24.0  666.0  20.2   

       11     12  
0  396.90  18.72  
1  395.38   3.11  
2  375.52   3.26  
3  396.90   8.01  
4  391.43  14.65  
      0
0  15.2
1  42.3
2  50.0
3  21.1
4  17.7


In [9]:
# split evaluation data into validation and test samples
from sklearn.model_selection import train_test_split
X_test, X_valid, y_test, y_valid = train_test_split(X_eval, y_eval, test_size = 0.5)

In [10]:
# normalize data using min-max scaling
# train, valid and test predictors are scaled using parameters of train predictors
# train and valid target values are scaled using parameters of train target 
# test target not scaled as it will be used for performance comparison on unseen data
# performance comparison to be done after unscaling predictions
y_train = y_train.reshape((-1, 1))
y_valid = y_valid.reshape((-1, 1))
scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
scaler_X.fit(X_train)
scaler_y.fit(y_train)
X_train_array = np.array(scaler_X.transform(X_train))
X_valid_array = np.array(scaler_X.transform(X_valid))
X_test_array = np.array(scaler_X.transform(X_test))
y_train_array = np.array(scaler_y.transform(y_train))
y_valid_array = np.array(scaler_y.transform(y_valid))

In [11]:
# check shape of validation data
print(X_valid_array.shape, y_valid_array.shape)

(51, 13) (51, 1)


In [12]:
# check test data shape
# test target should be rank-1 array, i.e., of the form: (N, )
print(type(X_test), type(y_test))
print(X_test.shape, y_test.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(51, 13) (51,)


In [13]:
# define model build
# define tunable hyperparameters using suitable hyperparameter types
# tunable hyperparameters in model: layers, neurons, dropout, learning rate
def build_model(hp):
  model = Sequential()
  model.add(Dense(units = hp.Int("dense_input", min_value = 32, max_value = 128, step = 32),
                                 input_shape = (X_train_array.shape[1],)))
  model.add(BatchNormalization())
  model.add(Activation("relu"))
  model.add(Dropout(hp.Float("drop_input", min_value = 0, max_value = 0.3, step = 0.1)))
  for i in range(hp.Int("num_intermediate_layers", 1, 3)):
    model.add(Dense(units = hp.Int(f'dense_{i}', min_value = 32, max_value = 128, step = 32)))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float(f'drop_{i}', min_value = 0, max_value = 0.3, step = 0.1)))
  model.add(Dense(1))
  model.compile(optimizer = Adam(learning_rate = hp.Float('lr', min_value = 0.0001, max_value = 0.01, sampling = "LOG", 
                                                          default = 0.01)), 
                loss = "mse", metrics = ["mae"])
  return(model)

In [14]:
## set up tensorboard
# clear logs from previous runs
!rm -rf ./logs/
# set up path where logs of tensorboard will be saved
# datetime suffix makes the logs unique
path = "logs/BH/" + datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
tensorboard = TensorBoard(log_dir = path, histogram_freq = 1) 

In [17]:
# select tuner for hyperparameter tuning (BayesianOptimization (BO) or Hyperband (HB))
tuner_choice = "HB"
# set max_trials (trials) for BO and max_epochs (ep) for HB
trials, ep = 20, 200
# set batch size
bs = 32

In [18]:
# set up tuner
if tuner_choice == "BO":
  tuner = BayesianOptimization(build_model, objective = "val_mae", max_trials = trials, executions_per_trial = 3, 
                               seed = 2020, project_name = "bo_tuning", overwrite = True)
else:
  tuner = Hyperband(build_model, objective = "val_mae", max_epochs = ep, executions_per_trial = 3,
                    seed = 2020, project_name = "hb_tuning", overwrite = True)

In [19]:
# tuner search space summary
print(tuner.search_space_summary())

None


In [20]:
# optimize hyperparameters
# we set up early stopping of tuning if validation loss doesn't improve after 5 epochs  
tuner.search(x = X_train_array, y = y_train_array, batch_size = bs, epochs = ep, verbose = 1, validation_data = (X_valid_array, y_valid_array), 
             callbacks = [EarlyStopping('val_loss', patience = 5), tensorboard])

Epoch 1/3
13/13 [==============================] - 0s 35ms/step - loss: 0.9918 - mae: 0.7729 - val_loss: 0.0797 - val_mae: 0.2225
Epoch 2/3
13/13 [==============================] - 0s 14ms/step - loss: 0.2493 - mae: 0.3912 - val_loss: 0.0583 - val_mae: 0.1836
Epoch 3/3
13/13 [==============================] - 0s 16ms/step - loss: 0.1311 - mae: 0.2778 - val_loss: 0.0441 - val_mae: 0.1573
Epoch 1/3
13/13 [==============================] - 0s 29ms/step - loss: 0.8368 - mae: 0.6872 - val_loss: 0.0980 - val_mae: 0.2586
Epoch 2/3
13/13 [==============================] - 0s 14ms/step - loss: 0.2388 - mae: 0.3739 - val_loss: 0.0527 - val_mae: 0.1730
Epoch 3/3
13/13 [==============================] - 0s 17ms/step - loss: 0.1354 - mae: 0.2745 - val_loss: 0.0382 - val_mae: 0.1416
Epoch 1/3
13/13 [==============================] - 0s 30ms/step - loss: 0.7954 - mae: 0.6936 - val_loss: 0.0933 - val_mae: 0.2624
Epoch 2/3
13/13 [==============================] - 0s 12ms/step - loss: 0.3000 - mae: 0.43

Epoch 1/3
13/13 [==============================] - 0s 32ms/step - loss: 0.8256 - mae: 0.6901 - val_loss: 0.0479 - val_mae: 0.1811
Epoch 2/3
13/13 [==============================] - 0s 14ms/step - loss: 0.3517 - mae: 0.4632 - val_loss: 0.0368 - val_mae: 0.1431
Epoch 3/3
13/13 [==============================] - 0s 16ms/step - loss: 0.2858 - mae: 0.4111 - val_loss: 0.0339 - val_mae: 0.1372
Epoch 1/3
13/13 [==============================] - 0s 30ms/step - loss: 1.5176 - mae: 0.8911 - val_loss: 0.0505 - val_mae: 0.1847
Epoch 2/3
13/13 [==============================] - 0s 15ms/step - loss: 0.3685 - mae: 0.4701 - val_loss: 0.0462 - val_mae: 0.1614
Epoch 3/3
13/13 [==============================] - 0s 18ms/step - loss: 0.3498 - mae: 0.4517 - val_loss: 0.0291 - val_mae: 0.1305
Epoch 1/3
13/13 [==============================] - 0s 33ms/step - loss: 0.9939 - mae: 0.7600 - val_loss: 0.0514 - val_mae: 0.1713
Epoch 2/3
13/13 [==============================] - 0s 12ms/step - loss: 0.5001 - mae: 0.53

Epoch 1/3
13/13 [==============================] - 0s 38ms/step - loss: 0.5407 - mae: 0.5065 - val_loss: 0.0833 - val_mae: 0.2514
Epoch 2/3
13/13 [==============================] - 0s 18ms/step - loss: 0.0662 - mae: 0.1938 - val_loss: 0.0322 - val_mae: 0.1312
Epoch 3/3
13/13 [==============================] - 0s 18ms/step - loss: 0.0349 - mae: 0.1367 - val_loss: 0.0324 - val_mae: 0.1289
Epoch 1/3
13/13 [==============================] - 0s 35ms/step - loss: 0.3915 - mae: 0.4266 - val_loss: 0.3098 - val_mae: 0.4494
Epoch 2/3
13/13 [==============================] - 0s 15ms/step - loss: 0.0484 - mae: 0.1613 - val_loss: 0.0949 - val_mae: 0.2304
Epoch 3/3
13/13 [==============================] - 0s 16ms/step - loss: 0.0274 - mae: 0.1250 - val_loss: 0.0343 - val_mae: 0.1467
Epoch 1/3
13/13 [==============================] - 0s 36ms/step - loss: 0.5964 - mae: 0.5115 - val_loss: 0.0352 - val_mae: 0.1601
Epoch 2/3
13/13 [==============================] - 0s 15ms/step - loss: 0.0633 - mae: 0.19

Epoch 1/3
13/13 [==============================] - 1s 40ms/step - loss: 0.9257 - mae: 0.7020 - val_loss: 0.0411 - val_mae: 0.1428
Epoch 2/3
13/13 [==============================] - 0s 14ms/step - loss: 0.3744 - mae: 0.4497 - val_loss: 0.0559 - val_mae: 0.1954
Epoch 3/3
13/13 [==============================] - 0s 15ms/step - loss: 0.3110 - mae: 0.4048 - val_loss: 0.0582 - val_mae: 0.2039
Epoch 1/3
13/13 [==============================] - 0s 36ms/step - loss: 1.1719 - mae: 0.8266 - val_loss: 0.0443 - val_mae: 0.1508
Epoch 2/3
13/13 [==============================] - 0s 17ms/step - loss: 0.4545 - mae: 0.5171 - val_loss: 0.0285 - val_mae: 0.1342
Epoch 3/3
13/13 [==============================] - 0s 15ms/step - loss: 0.2898 - mae: 0.4027 - val_loss: 0.0373 - val_mae: 0.1584
Epoch 1/3
13/13 [==============================] - 0s 36ms/step - loss: 1.2244 - mae: 0.8486 - val_loss: 0.0686 - val_mae: 0.2057
Epoch 2/3
13/13 [==============================] - 0s 15ms/step - loss: 0.4662 - mae: 0.49

Epoch 1/3
13/13 [==============================] - 0s 29ms/step - loss: 2.3979 - mae: 1.0957 - val_loss: 0.0456 - val_mae: 0.1607
Epoch 2/3
13/13 [==============================] - 0s 13ms/step - loss: 0.4803 - mae: 0.5198 - val_loss: 0.0373 - val_mae: 0.1575
Epoch 3/3
13/13 [==============================] - 0s 14ms/step - loss: 0.2942 - mae: 0.4067 - val_loss: 0.0315 - val_mae: 0.1312
Epoch 1/3
13/13 [==============================] - 0s 29ms/step - loss: 1.4664 - mae: 0.8981 - val_loss: 0.0216 - val_mae: 0.1190
Epoch 2/3
13/13 [==============================] - 0s 13ms/step - loss: 0.4501 - mae: 0.4980 - val_loss: 0.0419 - val_mae: 0.1578
Epoch 3/3
13/13 [==============================] - 0s 11ms/step - loss: 0.2207 - mae: 0.3501 - val_loss: 0.0249 - val_mae: 0.1203
Epoch 1/3
13/13 [==============================] - 0s 25ms/step - loss: 1.4489 - mae: 0.8891 - val_loss: 0.0511 - val_mae: 0.1782
Epoch 2/3
13/13 [==============================] - 0s 13ms/step - loss: 0.4277 - mae: 0.48

...

Epoch 1/200
13/13 [==============================] - 0s 34ms/step - loss: 1.2076 - mae: 0.8453 - val_loss: 0.0811 - val_mae: 0.2310
Epoch 2/200
13/13 [==============================] - 0s 16ms/step - loss: 0.3657 - mae: 0.4698 - val_loss: 0.0501 - val_mae: 0.1706
Epoch 3/200
13/13 [==============================] - 0s 15ms/step - loss: 0.1698 - mae: 0.3124 - val_loss: 0.0382 - val_mae: 0.1476
Epoch 4/200
13/13 [==============================] - 0s 13ms/step - loss: 0.0918 - mae: 0.2270 - val_loss: 0.0405 - val_mae: 0.1563
Epoch 5/200
13/13 [==============================] - 0s 16ms/step - loss: 0.0599 - mae: 0.1868 - val_loss: 0.0317 - val_mae: 0.1365
Epoch 6/200
13/13 [==============================] - 0s 11ms/step - loss: 0.0624 - mae: 0.1887 - val_loss: 0.0360 - val_mae: 0.1459
Epoch 7/200
13/13 [==============================] - 0s 12ms/step - loss: 0.0590 - mae: 0.1823 - val_loss: 0.0332 - val_mae: 0.1381
Epoch 8/200
13/13 [==============================] - 0s 13ms/step - loss: 0.

Epoch 1/200
13/13 [==============================] - 1s 40ms/step - loss: 1.2948 - mae: 0.9135 - val_loss: 0.1674 - val_mae: 0.3573
Epoch 2/200
13/13 [==============================] - 0s 17ms/step - loss: 0.7091 - mae: 0.6614 - val_loss: 0.1380 - val_mae: 0.3176
Epoch 3/200
13/13 [==============================] - 0s 18ms/step - loss: 0.6586 - mae: 0.6088 - val_loss: 0.1166 - val_mae: 0.2873
Epoch 4/200
13/13 [==============================] - 0s 18ms/step - loss: 0.5060 - mae: 0.5578 - val_loss: 0.1147 - val_mae: 0.2864
Epoch 5/200
13/13 [==============================] - 0s 18ms/step - loss: 0.4261 - mae: 0.5220 - val_loss: 0.1066 - val_mae: 0.2739
Epoch 6/200
13/13 [==============================] - 0s 17ms/step - loss: 0.3869 - mae: 0.4783 - val_loss: 0.0920 - val_mae: 0.2509
Epoch 7/200
13/13 [==============================] - 0s 16ms/step - loss: 0.2889 - mae: 0.4222 - val_loss: 0.0874 - val_mae: 0.2445
Epoch 8/200
13/13 [==============================] - 0s 18ms/step - loss: 0.

Epoch 1/200
13/13 [==============================] - 1s 39ms/step - loss: 1.7446 - mae: 1.0516 - val_loss: 0.1508 - val_mae: 0.3389
Epoch 2/200
13/13 [==============================] - 0s 18ms/step - loss: 1.0954 - mae: 0.8167 - val_loss: 0.1013 - val_mae: 0.2625
Epoch 3/200
13/13 [==============================] - 0s 18ms/step - loss: 0.9242 - mae: 0.7365 - val_loss: 0.0806 - val_mae: 0.2231
Epoch 4/200
13/13 [==============================] - 0s 17ms/step - loss: 0.9375 - mae: 0.7458 - val_loss: 0.0551 - val_mae: 0.1760
Epoch 5/200
13/13 [==============================] - 0s 17ms/step - loss: 0.9853 - mae: 0.7553 - val_loss: 0.0447 - val_mae: 0.1556
Epoch 6/200
13/13 [==============================] - 0s 18ms/step - loss: 0.8300 - mae: 0.6949 - val_loss: 0.0434 - val_mae: 0.1526
Epoch 7/200
13/13 [==============================] - 0s 17ms/step - loss: 0.8204 - mae: 0.6889 - val_loss: 0.0398 - val_mae: 0.1479
Epoch 8/200
13/13 [==============================] - 0s 17ms/step - loss: 0.

Epoch 1/200
13/13 [==============================] - 0s 35ms/step - loss: 1.9423 - mae: 1.1006 - val_loss: 0.1398 - val_mae: 0.3196
Epoch 2/200
13/13 [==============================] - 0s 17ms/step - loss: 1.1387 - mae: 0.7956 - val_loss: 0.1107 - val_mae: 0.2777
Epoch 3/200
13/13 [==============================] - 0s 15ms/step - loss: 0.8472 - mae: 0.7130 - val_loss: 0.0919 - val_mae: 0.2462
Epoch 4/200
13/13 [==============================] - 0s 15ms/step - loss: 0.7068 - mae: 0.6529 - val_loss: 0.0813 - val_mae: 0.2313
Epoch 5/200
13/13 [==============================] - 0s 15ms/step - loss: 0.6926 - mae: 0.6281 - val_loss: 0.0600 - val_mae: 0.1917
Epoch 6/200
13/13 [==============================] - 0s 16ms/step - loss: 0.6278 - mae: 0.6080 - val_loss: 0.0525 - val_mae: 0.1798
Epoch 7/200
13/13 [==============================] - 0s 17ms/step - loss: 0.5583 - mae: 0.5748 - val_loss: 0.0431 - val_mae: 0.1598
Epoch 8/200
13/13 [==============================] - 0s 18ms/step - loss: 0.

INFO:tensorflow:Oracle triggered exit


In [21]:
# tuning summary
print(tuner.results_summary())

None


In [22]:
# best hyperparameter values 
best_hp = pd.DataFrame(tuner.get_best_hyperparameters()[0].values, index = [0])
print(best_hp)

   dense_input  drop_input  num_intermediate_layers  dense_0  drop_0  \
0          128         0.0                        3       32     0.1   

         lr  dense_1  drop_1  dense_2  drop_2  tuner/epochs  \
0  0.000322       64     0.3       32     0.0           200   

   tuner/initial_epoch  tuner/bracket  tuner/round  
0                    0              0            0  


In [23]:
# best model
best_model = tuner.get_best_models()[0]
print(best_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0

In [24]:
# save best model
best_model.save(f'bh_model_{tuner_choice}.h5')

In [ ]:
# load best model (if not loaded)
best_model = keras.models.load_model('bh_model_HB.h5')

In [25]:
# predictions on test data
# predictions are not rank-1 arrays
# predictions to be flattened after inverse transformation
preds = best_model.predict(x = X_test_array)
print(type(preds), preds.shape)
print(pd.DataFrame(preds).head())

<class 'numpy.ndarray'> (51, 1)
          0
0  0.352468
1  0.314250
2  0.357416
3  0.647924
4  0.469309


In [26]:
# unscale predictions and convert into rank-1 array
preds_unscaled = np.array(scaler_y.inverse_transform(preds)).flatten()
print(type(preds_unscaled), preds_unscaled.shape)

<class 'numpy.ndarray'> (51,)


In [27]:
# compare unscaled predictions and actual values
print(pd.DataFrame(preds_unscaled, columns = ['predictions']).head())
print(pd.DataFrame(y_test, columns = ['actual_values']).head())

   predictions
0    20.861038
1    19.141232
2    21.083710
3    34.156597
4    26.118904
   actual_values
0           19.8
1           23.2
2           18.6
3           50.0
4           24.2


In [28]:
# performance on test data
mape = np.mean(np.abs((np.array(y_test) - np.array(preds_unscaled))/np.array(y_test))) * 100
mae = np.mean(np.abs(np.array(y_test) - np.array(preds_unscaled)))
print(f'mape = {mape} \n', f'mae = {mae}')

mape = 13.755827928278794 
 mae = 3.339658318313898


In [ ]:
# run tensorboard
%tensorboard --logdir logs/BH